In [ ]:
import json
import numpy as np
import nltk
nltk.download('punkt')
from nltk.stem.porter import PorterStemmer


import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader 


if __name__ == '__main__':
    stemmer = PorterStemmer()

    def tokenize(sentence):
        return nltk.word_tokenize(sentence)
    def stem(word):
        return stemmer.stem(word.lower())
    def bag_of_words(tokenized_sentence, all_words):
        tokenized_sentence = [stem(w) for w in tokenized_sentence]
        bag = np.zeros(len(all_words), dtype=np.float32)
        for idx,w in enumerate(all_words):
            if w in tokenized_sentence:
                bag[idx] = 1.0
        return bag
    class NeuralNet(nn.Module):
        def __init__(self, input_size, hidden_size, num_classes):
            super(NeuralNet, self).__init__()
            #linear layers for hidden layers of neural network
            self.l1 = nn.Linear(input_size, hidden_size)
            self.l2 = nn.Linear(hidden_size, hidden_size)
            self.l3 = nn.Linear(hidden_size, num_classes)
            #activation function for in between the layers
            self.relu = nn.ReLU()
        
        def forward(self,x):
            out = self.l1(x)
            out = self.relu(out)
            out = self.l2(out)
            out = self.relu(out)
            out = self.l3(out)
            return out



    with open('intents.json','r') as f:
        intents = json.load(f)

    all_words = []
    tags = []
    xy = []
    for intent in intents['intents']:
        tag = intent['tag']
        tags.append(tag)
        for pattern in intent['patterns']:
            w = tokenize(pattern)
            all_words.extend(w)
            xy.append((w,tag))

    ignore_words = ['?', '!', '.', ',']
    all_words = [stem(w) for w in all_words if w not in ignore_words]
    all_words = sorted(set(all_words))
    tags = sorted(set(tags))

    X_train = []
    Y_train = []

    for (pattern_sentence, tag) in xy:
        bag = bag_of_words(pattern_sentence, all_words)
        X_train.append(bag)

        label = tag.index(tag)
        Y_train.append(label)

    X_train = np.array(X_train)
    Y_train = np.array(Y_train)

    class ChatDataset(Dataset):
        def __init__(self):
            self.n_samples = len(X_train)
            self.x_data = X_train
            self.y_data = Y_train

        def __getitem__(self, index):
            return self.x_data[index], self.y_data[index]

        def __len__(self):
            return self.n_samples
        
    #Hyperparameters
    batch_size = 8
    hidden_size = 8
    output_size = len(tags)
    input_size = len(X_train[0])
    learning_rate = 0.001
    num_epochs = 1000

    dataset = ChatDataset()

    train_loader = DataLoader(dataset = dataset, batch_size = batch_size, shuffle = True, num_workers = 2)

    device = torch.device('cuda'if torch.cuda.is_available() else 'cpu')
    model = NeuralNet(input_size, hidden_size, output_size).to(device)


    #loss and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

    for epoch in range(num_epochs):
        for (words, labels) in train_loader:
            words = words.to(device)
            labels = labels.to(device)
            outputs = model(words)
            loss = criterion(outputs, labels)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        if (epoch + 1) % 100 == 0:
            print(f'epoch {epoch+1}/{num_epochs}, loss = {loss.item(): .4f}')    

    print(f'final loss, loss = {loss.item(): .4f}')   

    data = {
        "model_state": model.state_dict(),
        "input_size": input_size,
        "output_size": output_size,
        "hidden_size": hidden_size,
        "all_words": all_words,
        "tags": tags
    }

    FILE = "data.pth"
    torch.save(data, FILE)

    print(f'training complete. file saved to {FILE}')



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


epoch 100/1000, loss =  0.0232
epoch 200/1000, loss =  0.0020
epoch 300/1000, loss =  0.0001
epoch 400/1000, loss =  0.0007
epoch 500/1000, loss =  0.0005
epoch 600/1000, loss =  0.0000
epoch 700/1000, loss =  0.0000
epoch 800/1000, loss =  0.0000
epoch 900/1000, loss =  0.0001
epoch 1000/1000, loss =  0.0000
final loss, loss =  0.0000
training complete. file saved to data.pth


In [20]:
import random
import json
import torch

def tokenize(sentence):
    return nltk.word_tokenize(sentence)
def stem(word):
    return stemmer.stem(word.lower())
def bag_of_words(tokenized_sentence, all_words):
    tokenized_sentence = [stem(w) for w in tokenized_sentence]
    bag = np.zeros(len(all_words), dtype=np.float32)
    for idx,w in enumerate(all_words):
        if w in tokenized_sentence:
            bag[idx] = 1.0
    return bag
class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        #linear layers for hidden layers of neural network
        self.l1 = nn.Linear(input_size, hidden_size)
        self.l2 = nn.Linear(hidden_size, hidden_size)
        self.l3 = nn.Linear(hidden_size, num_classes)
        #activation function for in between the layers
        self.relu = nn.ReLU()
    
    def forward(self,x):
        out = self.l1(x)
        out = self.relu(out)
        out = self.l2(out)
        out = self.relu(out)
        out = self.l3(out)
        return out

device = torch.device('cpu')
with open('intents.json','r') as f:
    intents = json.load(f)

FILE = "data.pth"
data = torch.load(FILE)

input_size = data["input_size"]
hidden_size = data["hidden_size"]
output_size = data["output_size"]
all_words = data["all_words"]
tags = data["tags"]
model_state = data["model_state"]
model = NeuralNet(input_size, hidden_size, output_size). to(device)
model.load_state_dict(model_state)
model.eval()

bot_name = "Baymax"
print("Let's chat! type 'quit' to exit")
while(True):
  sentence = input('You: ')
  if sentence == "quit":
    break
  sentence = tokenize(sentence)
  X = bag_of_words(sentence, all_words)
  X = X.reshape(1, X.shape[0])
  X = torch.from_numpy(X)
  output = model(X)
  _, predicted = torch.max(output, dim = 1)
  tag = tags[predicted.item()]

  probs = torch.softmax(output, dim = 1)
  prob = probs[0][predicted.item()]

  if prob.item() > 0.75:
    for intent in intents["intents"]:
      if tag == intent["tag"]:
        print(f'{bot_name}: {random.choice(intent["responses"])}')
  else:
    print(f'{bot_name}: I do not understand...')
  

Let's chat! type 'quit' to exit
You: HELLO
Baymax: Shipping takes 2-4 days
You: QUIT
Baymax: Shipping takes 2-4 days
You: quit
